In [54]:
import numpy as np
import matplotlib.pyplot as plt

def inicializar_grilla(n, p):
    """
    Inicializa una grilla bidimensional con celdas vacías, azules y rojas según las probabilidades proporcionadas.

    Args:
        n (int): Tamaño de la grilla (n x n).
        p (list): Lista de probabilidades [p_vacía, p_azul, p_roja].

    Returns:
        np.ndarray: Grilla inicializada.
    """
    grilla = np.random.choice([0, 1, 2], size=(n, n), p=p)
    return grilla

def contar_vecinos(grilla, i, j):
    """
    Cuenta los vecinos de una celda dada en una grilla.

    Args:
        grilla (np.ndarray): Grilla bidimensional.
        i (int): Índice de fila de la celda.
        j (int): Índice de columna de la celda.

    Returns:
        list: Lista de valores de los vecinos.
    """
    vecinos = []
    n = grilla.shape[0]
    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:
            if di == 0 and dj == 0:
                continue
            ni, nj = i + di, j + dj
            if 0 <= ni < n and 0 <= nj < n:
                vecinos.append(grilla[ni, nj])
    return vecinos

def calcular_felicidad(grilla, i, j, t):
    """
    Calcula si una celda dada está feliz o infeliz según la tolerancia t.

    Args:
        grilla (np.ndarray): Grilla bidimensional.
        i (int): Índice de fila de la celda.
        j (int): Índice de columna de la celda.
        t (int): Tolerancia.

    Returns:
        bool: True si la celda está feliz, False si está infeliz.
    """
    vecinos = contar_vecinos(grilla, i, j)
    color_propio = grilla[i, j]
    mismo_color = vecinos.count(color_propio)
    color_distinto = len(vecinos) - mismo_color
    if color_distinto >= t:
        return False
    else:
        return True

def encontrar_celdas_infelices(grilla, t):
    """
    Encuentra todas las celdas infelices en la grilla según la tolerancia t.

    Args:
        grilla (np.ndarray): Grilla bidimensional.
        t (int): Tolerancia.

    Returns:
        list: Lista de índices de celdas infelices.
    """
    celdas_infelices = []
    n = grilla.shape[0]
    for i in range(n):
        for j in range(n):
            if grilla[i, j] != 0 and not calcular_felicidad(grilla, i, j, t):
                celdas_infelices.append(i * n + j)  # Convertir a índice unidimensional
    return celdas_infelices

def intercambiar_celdas(grilla, celda1, celda2):
    """
    Intercambia los colores de dos celdas en la grilla.

    Args:
        grilla (np.ndarray): Grilla bidimensional.
        celda1 (int): Índice de la primera celda.
        celda2 (int): Índice de la segunda celda.

    Returns:
        np.ndarray: Grilla con las celdas intercambiadas.
    """
    i1, j1 = divmod(celda1, grilla.shape[0])  # Convertir de índice unidimensional a (i, j)
    i2, j2 = divmod(celda2, grilla.shape[0])
    grilla[i1, j1], grilla[i2, j2] = grilla[i2, j2], grilla[i1, j1]
    return grilla

def visualizar_grilla(grilla, paso):
    """
    Visualiza la grilla en una representación gráfica y guarda la imagen.

    Args:
        grilla (np.ndarray): Grilla bidimensional.
        paso (int): El número del paso actual.
    """
    cmap = plt.get_cmap('coolwarm', 3)
    plt.imshow(grilla, cmap=cmap, vmin=0, vmax=2)
    plt.axis('off')
    plt.close()  # Cerrar la figura

def modelo_sakoda_schelling(n, p, t, pasos):
    """
    Ejecuta el modelo de Sakoda/Schelling y devuelve una lista de grillas.

    Args:
        n (int): Tamaño de la grilla (n x n).
        p (list): Lista de probabilidades [p_vacía, p_azul, p_roja].
        t (int): Tolerancia.
        pasos (int): Número de pasos a ejecutar.

    Returns:
        list: Lista de grillas.
    """
    grilla = inicializar_grilla(n, p)
    grillas = [grilla.copy()]  # Agregar la grilla inicial a la lista de grillas
    
    for _ in range(pasos):
        celdas_infelices = encontrar_celdas_infelices(grilla, t)
        if len(celdas_infelices) == 0:
            break
        
        celda1, celda2 = np.random.choice(celdas_infelices, size=2, replace=False)
        grilla = intercambiar_celdas(grilla, celda1, celda2)
        
        grillas.append(grilla.copy())  # Agregar la grilla actualizada a la lista de grillas
    
    return grillas

In [84]:
# Parámetros de entrada
n = 40  # Tamaño de la grilla (n x n)
p = [0.1, 0.45, 0.45]  # Probabilidad de celdas vacías, azules y rojas respectivamente
t = 2  # Tolerancia
pasos = 100  # Número de pasos

# Ejecutar el modelo de Sakoda/Schelling
automata = modelo_sakoda_schelling(n, p, t, pasos)

In [85]:
import matplotlib.colors as mcolors

def crear_gif(automata, n, pasos, p):
    nombre_archivo = f"SAKODA_size{n}_pasos{pasos}_dist{p}.gif"

    # Crear un mapa de colores personalizado
    cmap = mcolors.ListedColormap(['white', 'blue', 'red'])

    imagenes = []
    for idx, matriz in enumerate(automata):
        imagen = plt.imshow(matriz, cmap=cmap, interpolation='nearest', vmin=0, vmax=2)
        titulo = f"Generación {idx} \nDistr (vacio,azul,rojo) {p}"
        plt.title(titulo)
        plt.axis('on')
        filename = "temp.png"
        plt.savefig(filename)
        plt.close()

        image = imageio.v2.imread(filename)
        os.remove(filename)
        imagenes.append(image)

    imageio.mimsave(nombre_archivo, imagenes, fps=10)

In [86]:
# Crear el GIF
crear_gif(automata, n, pasos, p)